In [1]:
# Импортируем библиотеки
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

#### **Задача:** выявить особенности датасета, которые привели ухудшению качества моделей и объяснить причины почему так происходило.

**Известные данные:**

- Первая колонка представляет собой индекс.
- Вторая колонка (request) - это идентификатор запроса.
- Третья колонка (object) - это идентификатор объекта, на который кликнул пользователь.
- Четвертая колонка (label) - это метка, где 1 указывает на положительный пример, а 0 - на негативный пример.

In [2]:
# Задаем изначальные названия столбцов
column_names = ['request', 'object', 'label']
df = pd.read_csv('~/Applications/Test-tasks-for-companies/Test tasks for 2GIS/Test/clicks_dataset_msk_20230101_20230725_spec.csv', names=column_names)

print('Размер датасета:', df.shape)

Размер датасета: (12979882, 3)


In [3]:
# Вывод первых нескольких строк датасета для знакомства с данными
df.head(4)

,request,object,label
0,1590973,168299,1
1,1590973,718560,0
2,1234953,325828,1
3,1234953,135968,0


На основе известной информации о данных, посмотрим на пример:

- Строка 0: Пользователь сделал запрос с идентификатором 1590973 и кликнул на объект с идентификатором 168299, что соответствует положительному примеру (метка 1).
- Строка 1: Для того же запроса 1590973 был подобран негативный объект с идентификатором 718560 (метка 0) из другой рубрики.
- Строка 2: Пользователь сделал запрос с идентификатором 1234953 и кликнул на объект с идентификатором 325828, что соответствует положительному примеру (метка 1).
- Строка 3: Для того же запроса 1234953 был подобран негативный объект с идентификатором 135968 (метка 0) из другой рубрики.

Теперь, когда мы понимаем структуру и содержание данных, мы можем перейти к более детальному анализу и выявлению особенностей, которые могут повлиять на качество модели при тестировании. 

In [4]:
# Проверка наличия пропущенных значений в каждом столбце
missing_values_per_column = df.isnull().sum()
print("Количество пропущенных значений в каждом столбце:")
print(missing_values_per_column)

Количество пропущенных значений в каждом столбце:
request    0
object     0
label      0
dtype: int64


Пропущенных значений в данных нет, но, возможно, в исходных данных **есть дубликаты строк**, которые нужно удалить.

In [5]:
# Удаление дубликатов из датафрейма
df_without_duplicates = df.drop_duplicates()

# Вывод информации о новом датафрейме без дубликатов
print("Размер датафрейма до удаления дубликатов:", df.shape)
print("Размер датафрейма после удаления дубликатов:", df_without_duplicates.shape)

Размер датафрейма до удаления дубликатов: (12979882, 3)
Размер датафрейма после удаления дубликатов: (12977297, 3)


Из предоставленного описания данных следует, что для каждого запроса должны быть ровно две строки:

- Одна строка с положительным примером (метка 1).
- Одна строка с негативным примером (метка 0).

**Если у запроса больше двух строк или запросу отсутствует одна из строк (либо положительная, либо негативная), то это может считаться ошибкой в данных или несоответствием описанию.**

In [6]:
# Чтобы вывести только те строки в датафрейме, где значение value_counts равно 2

# Выборка строк с value_counts == 2
df_value_counts_2 = df_without_duplicates[df_without_duplicates['request'].map(df_without_duplicates['request'].value_counts()) == 2].reset_index(drop=True)

# Подсчет строк, где value_counts != 2
count_not_2 = len(df_without_duplicates) - len(df_value_counts_2)

print("Количество строк с value_counts == 2:", len(df_value_counts_2))
print("Количество строк с value_counts != 2:", count_not_2)

Количество строк с value_counts == 2: 8394858
Количество строк с value_counts != 2: 4582439


Проблемой могут быть данные, в которых уникальному идентификатору запроса будет соответствовать совпадающий уникальный идентификатор объекта для разных меткок. В случае, если такие данные обнаружатся, удалим данные о таких запросах.

In [7]:
# Проверка, есть ли строки, где номер запроса соответствует номеру объекта для разных меток
mismatched_rows = df_value_counts_2['request'] == df_value_counts_2['object']

# Проверка, есть ли хотя бы одна такая строка
if mismatched_rows.any():
    print("Есть строки, где номер запроса соответствует номеру объекта для разных меток.")
else:
    print("Нет строк, где номер запроса соответствует номеру объекта для разных меток.")

Есть строки, где номер запроса соответствует номеру объекта для разных меток.


In [8]:
# Получение списка уникальных запросов, где номер запроса соответствует номеру объекта для разных меток
matching_requests = df_value_counts_2.loc[df_value_counts_2['request'] == df_value_counts_2['object'], 'request'].unique()

# Вывод списка запросов, удовлетворяющих условию
print("Запросы, где номер запроса соответствует номеру объекта для разных меток:")
print(matching_requests)

# Удаление всех строк, связанных с этими запросами
df_without_matching = df_value_counts_2[~df_value_counts_2['request'].isin(matching_requests)].reset_index(drop=True)

# Вывод информации о новом датафрейме без этих строк
print("Размер датафрейма до удаления строк:", df_value_counts_2.shape)
print("Размер датафрейма после удаления строк:", df_without_matching.shape)

Запросы, где номер запроса соответствует номеру объекта для разных меток:
[930644]
Размер датафрейма до удаления строк: (8394858, 3)
Размер датафрейма после удаления строк: (8394856, 3)


Хотелось бы уточнить, могут ли уникальные номера запросов совпадать с уникальными номерами объектов, так как иногда такая тенденция может нарушить требования консистентности данных. В случае, если таких пересечений не должно быть и они добавлены по ошибке, тогда их следовало бы удалить.

In [9]:
len(set(df_without_matching['object']) & set(df_without_matching['request']))

965954

Дополнительно проверим на соответствие одному запросу меток 1 и 0.

In [10]:
# Группировка данных по запросам и подсчет уникальных меток
grouped_data = df_without_matching.groupby('request')['label'].nunique()

# Проверка соответствия одного запроса меткам 1 и 0
check = grouped_data[(grouped_data != 2)]

if not check.empty:
    print("Найдены запросы, которые не соответствуют требованию наличия как минимум одной строки с меткой 1 и одной строкой с меткой 0.")
    print("Нарушения обнаружены для следующих запросов:")
    print(check)
else:
    print("Все запросы соответствуют требованию наличия как минимум одной строки с меткой 1 и одной строкой с меткой 0.")

Все запросы соответствуют требованию наличия как минимум одной строки с меткой 1 и одной строкой с меткой 0.


Также проверим может ли быть такая ситуация, что одному уникальному идентификатору запроса будет соответствовать один уникальный идентификатор объекта.

In [11]:
# До этого мы проверяли по количеству, проверим еще на уникальность
# Подсчет количества уникальных объектов для каждого уникального запроса
unique_objects_per_request = df_without_matching.groupby('request')['object'].nunique()

# Найти запросы, для которых количество уникальных объектов равно 1
requests_with_single_object = unique_objects_per_request[unique_objects_per_request == 1]

# Вывод найденных запросов
print("Для некоторых уникальных запросов соответствует только один объект:")
print(requests_with_single_object)

Для некоторых уникальных запросов соответствует только один объект:
Series([], Name: object, dtype: int64)


После проверки того, соответствуют ли всем уникальным запросам разные объекты, можно сделать вывод, что в датасете **нет повторений объектов для одного и того же запроса**.

In [12]:
# Общая информация о датасете: количество строк и столбцов, типы данных
print(df_without_matching.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8394856 entries, 0 to 8394855
Data columns (total 3 columns):
 #   Column   Dtype
---  ------   -----
 0   request  int64
 1   object   int64
 2   label    int64
dtypes: int64(3)
memory usage: 192.1 MB
None


In [13]:
# Посмотрим на распределение меток классов (0 и 1) в датасете, чтобы определить, есть ли дисбаланс
class_distribution = df_without_matching['label'].value_counts()
print("Class distribution:")
print(class_distribution)

Class distribution:
1    4197428
0    4197428
Name: label, dtype: int64


Распределение меток классов в датафрейме показывает, что количество положительных (метка 1) и негативных (метка 0) примеров одинаково, что указывает на **отсутствие дисбаланса классов**. Это положительно сказывается на возможности модели обучаться и делать прогнозы для обоих классов с равной точностью.

**Из анализа поставленной задачи можно сделать следующие выводы относительно возможных причин, влияющих на низкое качество:**

- Некоторые запросы в данных не имеют полной пары строк с метками 1 и 0, что нарушает условие обучения моделей, требующее две строки для каждого запроса: одну с меткой 1 (положительный пример, когда пользователь кликнул на объект) и одну с меткой 0 (негативный пример, когда пользователь не кликнул на объект).
- Следует обратить внимание на то, что некоторые уникальные идентификаторы запросов совпадают с уникальными идентификаторами объектов, что может указывать на некорректное представление данных или наличие ошибок в сборе данных.
- В датасете были обнаружены дубликаты, что может искажать результаты анализа и моделирования данных.
- В датасете был обнаружен номер запроса, который соответствовал номеру объекта для разных меток.